# Intro

This is a second part of an End-to-End ML project. I'll use [Online Retail Data Set](https://www.kaggle.com/datasets/carrie1/ecommerce-data) to do RFM Analysis of customers. The dataset contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. The stages here are the following:

* Data Preparation
* RFM Analysis

But first, let's import libraries and data.

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('../data/data.csv', encoding="ISO-8859-1")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Data Preparation

Here let's take a look at the data and clean up a bit. As I've already explored the data in first part, here I'll go faster with less comments.

In [4]:
df.shape

(541909, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


There are lots of nulls in `CustomerID`, let's remove rows records of these customers.

In [6]:
df.dropna(subset='CustomerID', inplace=True)
df.shape

(406829, 8)

In [7]:
df=df[df['Quantity']>0]
df.shape

(397924, 8)

In [8]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])

What about number of Products, Transactions and Customers?

In [9]:
print(f'No. of Products: {len(df["StockCode"].value_counts())}')
print(f'No. of Transactions: {len(df["InvoiceNo"].value_counts())}')
print(f'No. of Customers: {len(df["CustomerID"].value_counts())}')

No. of Products: 3665
No. of Transactions: 18536
No. of Customers: 4339


# RFM Analysis

It's time to perform [RFM Analysis](https://www.blastanalytics.com/blog/rfm-analysis-boosts-sales). RFM stands for Recency, Frequency, and Monetary value, each corresponding to some key customer trait. These RFM metrics are important indicators of a customer’s behavior because frequency and monetary value affects a customer’s lifetime value, and recency affects retention, a measure of engagement.
RFM factors illustrate these facts:
* The more recent the purchase, the more responsive the customer is to promotions
* The more frequently the customer buys, the more engaged and satisfied they are
* Monetary value differentiates heavy spenders from low-value purchasers

## Recency

To calculate recency, we need to choose a date point from which we evaluate how many days ago was the customer's last purchase. This date will be `2011-12-11`.

In [10]:
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

Let's add `Date` column for later use.

In [11]:
df['Date']=df['InvoiceDate'].dt.date
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01


In [12]:
today=dt.date(2011, 12, 11)

rfm_df=df.groupby('CustomerID', as_index=False)['Date'].max()
rfm_df['Recency']=rfm_df['Date'].apply(lambda x: (today-x).days)
rfm_df.drop(columns=['Date'], inplace=True)
rfm_df.head()

,CustomerID,Recency
0,12346.0,327
1,12347.0,4
2,12348.0,77
3,12349.0,20
4,12350.0,312


## Frequency

Frequency helps us to know how many times a customer purchased from us. To do that we need to check how many unique invoices are registered by the same customer.

In [13]:
freq=df.groupby('CustomerID', as_index=False)['InvoiceNo'].nunique()['InvoiceNo']
rfm_df['Frequency']=freq
rfm_df.head()

,CustomerID,Recency,Frequency
0,12346.0,327,1
1,12347.0,4,7
2,12348.0,77,4
3,12349.0,20,1
4,12350.0,312,1


## Monetary

Monetary attribute answers the question: How much money did the customer spent over time? But first a new column called `TotalPrice` must be added.

In [14]:
df['TotalCost']=df['Quantity']*df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Date,TotalCost
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34


In [15]:
monetary=df.groupby('CustomerID', as_index=False)['TotalCost'].sum()['TotalCost']
rfm_df['Monetary']=monetary
rfm_df.head()

,CustomerID,Recency,Frequency,Monetary
0,12346.0,327,1,77183.60
1,12347.0,4,7,4310.00
2,12348.0,77,4,1797.24
3,12349.0,20,1,1757.55
4,12350.0,312,1,334.40


That's it, we have Recency, Frequency and Monetary. Now we can calculate RFM scores. But first, I want to show a one-cell solution that brings us the same results.

In [16]:
today_date = dt.datetime(2011,12,11)

rfm = df.groupby('CustomerID').agg({'InvoiceDate': lambda invoice_date: (today_date - invoice_date.max()).days,
                                    'InvoiceNo': lambda invoice: invoice.nunique(),
                                    'TotalCost': lambda total_price: total_price.sum()})

rfm = rfm.reset_index()
rfm.rename(columns={'InvoiceDate':'recency',
                    'InvoiceNo':'frequency',
                    'TotalCost':'monetary'}, inplace=True)
rfm.head()

,CustomerID,recency,frequency,monetary
0,12346.0,326,1,77183.60
1,12347.0,3,7,4310.00
2,12348.0,76,4,1797.24
3,12349.0,19,1,1757.55
4,12350.0,311,1,334.40


## RFM Scores

Now we need to get individual R, F and M scores and later combine them to one metric. Individual scores have range 1-4, 1 is the best, 4 is the worst. Foe example:
* R=1-->very recent pruchases
* F=1-->very frequent pruchases
* M=1-->spends a lot
First, we divide the data in Scores.

In [17]:
quantiles=rfm_df.quantile(q=[0.25,0.5,0.75]).drop(columns=['CustomerID'])
quantiles

,Recency,Frequency,Monetary
0.25,19.0,1.0,307.245
0.50,52.0,2.0,674.450
0.75,143.5,5.0,1661.640


In [18]:
quantiles_dict=quantiles.to_dict()
quantiles_dict

{'Recency': {0.25: 19.0, 0.5: 52.0, 0.75: 143.5},
 'Frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 5.0},
 'Monetary': {0.25: 307.245, 0.5: 674.45, 0.75: 1661.64}}

Now we define two functions to segment the data using these Scores.

In [19]:
def r_score(x, p, d):
    if x<=d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

def f_s_score(x, p, d):
    if x<=d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [20]:
rfm_df['R_Score']=rfm_df['Recency'].apply(lambda x: r_score(x, 'Recency', quantiles_dict))
rfm_df['F_Score']=rfm_df['Frequency'].apply(lambda x: f_s_score(x, 'Frequency', quantiles_dict))
rfm_df['M_Score']=rfm_df['Monetary'].apply(lambda x: f_s_score(x, 'Monetary', quantiles_dict))

rfm_df.head()

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score
0,12346.0,327,1,77183.60,4,4,1
1,12347.0,4,7,4310.00,1,1,1
2,12348.0,77,4,1797.24,3,2,1
3,12349.0,20,1,1757.55,2,4,1
4,12350.0,312,1,334.40,4,4,3


Great! Now we just need to concatenate these scores into one RFM score.

In [21]:
rfm_df['RFM_Score']=rfm_df['R_Score'].astype(str)+rfm_df['F_Score'].astype(str)+rfm_df['M_Score'].astype(str)

rfm_df.head()

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
0,12346.0,327,1,77183.60,4,4,1,441
1,12347.0,4,7,4310.00,1,1,1,111
2,12348.0,77,4,1797.24,3,2,1,321
3,12349.0,20,1,1757.55,2,4,1,241
4,12350.0,312,1,334.40,4,4,3,443


Perfect! Now we just need to interpret thse scores.



| Segment | RFM | Description | Marketing |
| --- | --- | --- | --- |
|Best Customers |111|Bought most recently and most often, and spend the most|No price incentives, new products, and loyalty programs|
|Loyal Customers|X1X|	Buy most frequently|	Use R and M to further segment|
|Big Spenders	|XX1	|Spend the most	|Market your most expensive products|
|Almost Lost	|311|	Haven’t purchased for some time, but purchased frequently and spend the most|	Aggressive price incentives|
|Lost Customers	|411|	Haven’t purchased for some time, but purchased frequently and spend the most|	Aggressive price incentives|
|Lost Cheap Customers	|444|	Last purchased long ago, purchased few, and spent little|	Don’t spend too much trying to re-acquire|

And finally let's see how many customers are there in each category.

In [22]:
print("Best Customers: ",len(rfm_df[rfm_df['RFM_Score']=='111']))
print('Loyal Customers: ',len(rfm_df[rfm_df['F_Score']==1]))
print("Big Spenders: ",len(rfm_df[rfm_df['M_Score']==1]))
print('Almost Lost: ', len(rfm_df[rfm_df['RFM_Score']=='311']))
print('Lost Customers: ',len(rfm_df[rfm_df['RFM_Score']=='411']))
print('Lost Cheap Customers: ',len(rfm_df[rfm_df['RFM_Score']=='444']))

Best Customers:  456
Loyal Customers:  872
Big Spenders:  1085
Almost Lost:  70
Lost Customers:  10
Lost Cheap Customers:  443


In [23]:
rfm_df[rfm_df['RFM_Score']=='111']

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
1,12347.0,4,7,4310.00,1,1,1,111
15,12362.0,5,10,5226.23,1,1,1,111
34,12388.0,17,6,2780.66,1,1,1,111
56,12417.0,5,9,3649.10,1,1,1,111
61,12423.0,2,8,1859.31,1,1,1,111
...,...,...,...,...,...,...,...,...
4299,18230.0,11,7,2810.20,1,1,1,111
4308,18241.0,11,17,2073.09,1,1,1,111
4310,18245.0,9,7,2567.06,1,1,1,111
4328,18272.0,4,6,3078.58,1,1,1,111


That's all! Now, both Segmentation and RFM are ready. Along with some EDA, they both will be used in a Web App. Hope you liked the project and learned something new:)